In [1]:
import pymysql.cursors
import pandas as pd

In [2]:
# MySQL DB에 저장된 데이터 확인

conn = pymysql.connect(host='127.0.0.1', user='root', 
                       password='0000', db='TIP_Schema', charset='utf8',
                       autocommit=True, cursorclass=pymysql.cursors.DictCursor)

try:
    with conn.cursor() as curs:
        sql = "select * from TIP_Schema.tips;"
        curs.execute(sql)
        rs = curs.fetchall()
        df = pd.DataFrame(rs)
        print(df)
        df.to_csv('tips2.csv')
        
finally: 
    conn.close()

     total_bill   tip     sex smoker   day    time  size
0         16.99  1.01  Female     No   Sun  Dinner   2.0
1         10.34  1.66    Male     No   Sun  Dinner   3.0
2         21.01  3.50    Male     No   Sun  Dinner   3.0
3         23.68  3.31    Male     No   Sun  Dinner   2.0
4         24.59  3.61  Female     No   Sun  Dinner   4.0
..          ...   ...     ...    ...   ...     ...   ...
243       18.78  3.00  Female     No  Thur  Dinner   2.0
244       16.99   NaN  Female     No   Sun  Dinner   2.0
245       20.34  1.66    None     No   Sun  Dinner   3.0
246       13.23  2.66    Male    Yes   Sat    None   NaN
247       26.34  2.20  Female     No   Fri   Lunch   4.0

[248 rows x 7 columns]


MySQL에 저장된 데이터를 추출하여 pandas로 읽고, 
## 문자형 변수를 숫자형 변수로 변환.

In [3]:
tips = pd.read_csv('tips2.csv', index_col=0)
tips

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2.0
1,10.34,1.66,Male,No,Sun,Dinner,3.0
2,21.01,3.50,Male,No,Sun,Dinner,3.0
3,23.68,3.31,Male,No,Sun,Dinner,2.0
4,24.59,3.61,Female,No,Sun,Dinner,4.0
...,...,...,...,...,...,...,...
243,18.78,3.00,Female,No,Thur,Dinner,2.0
244,16.99,NaN,Female,No,Sun,Dinner,2.0
245,20.34,1.66,NaN,No,Sun,Dinner,3.0
246,13.23,2.66,Male,Yes,Sat,NaN,NaN


In [4]:
# 변수 형태 변환
tips['sex'].replace({'Female': 0, 'Male':1}, inplace=True)
tips['smoker'].replace({'No': 0, 'Yes':1}, inplace=True)
tips['day'].replace({'Thur': 0, 'Fri':1, 'Sat': 2, 'Sun': 3}, inplace=True)
tips['time'].replace({'Lunch': 0, 'Dinner':1}, inplace=True)

''' inplace=True 대신 사용 가능.
tips['sex'] = tips['sex'].replace({'Female': 0, 'Male':1})
tips['smoker'] = tips['smoker'].replace({'No': 0, 'Yes':1})
tips['day'] = tips['day'].replace({'Thur': 0, 'Fri':1, 'Sat': 2, 'Sun': 3})
tips['time'] = tips['time'].replace({'Lunch': 0, 'Dinner':1})
'''

tips.tail(5)

,total_bill,tip,sex,smoker,day,time,size
243,18.78,3.00,0.0,0,0,1.0,2.0
244,16.99,NaN,0.0,0,3,1.0,2.0
245,20.34,1.66,NaN,0,3,1.0,3.0
246,13.23,2.66,1.0,1,2,NaN,NaN
247,26.34,2.20,0.0,0,1,0.0,4.0


In [5]:
# 각 변수가 숫자형으로 변환된 것을 확인.
tips.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 248 entries, 0 to 247
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   total_bill  248 non-null    float64
 1   tip         247 non-null    float64
 2   sex         247 non-null    float64
 3   smoker      248 non-null    int64  
 4   day         248 non-null    int64  
 5   time        247 non-null    float64
 6   size        247 non-null    float64
dtypes: float64(5), int64(2)
memory usage: 15.5 KB


## 결측치 제거

In [6]:
# 결측치 확인
tips.isnull().sum()

total_bill    0
tip           1
sex           1
smoker        0
day           0
time          1
size          1
dtype: int64

In [7]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

In [8]:
tips.describe()

,total_bill,tip,sex,smoker,day,time,size
count,248.000000,247.000000,247.000000,248.000000,248.000000,247.000000,247.000000
mean,19.776895,2.988259,0.639676,0.379032,1.733871,0.720648,2.574899
std,8.851566,1.378892,0.481069,0.486127,1.153125,0.449592,0.950747
min,3.070000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,13.347500,2.000000,0.000000,0.000000,0.750000,0.000000,2.000000
50%,17.795000,2.830000,1.000000,0.000000,2.000000,1.000000,2.000000
75%,24.127500,3.530000,1.000000,1.000000,3.000000,1.000000,3.000000
max,50.810000,10.000000,1.000000,1.000000,3.000000,1.000000,6.000000


In [9]:
type(tips)

pandas.core.frame.DataFrame

In [10]:
# DataFrame 형태의 자료를 array로 변환 - numpy의 isnan을 활용하기 위해.
data = tips.values
data

array([[16.99,  1.01,  0.  , ...,  3.  ,  1.  ,  2.  ],
       [10.34,  1.66,  1.  , ...,  3.  ,  1.  ,  3.  ],
       [21.01,  3.5 ,  1.  , ...,  3.  ,  1.  ,  3.  ],
       ...,
       [20.34,  1.66,   nan, ...,  3.  ,  1.  ,  3.  ],
       [13.23,  2.66,  1.  , ...,  2.  ,   nan,   nan],
       [26.34,  2.2 ,  0.  , ...,  1.  ,  0.  ,  4.  ]])

In [11]:
type(data)

numpy.ndarray

In [12]:
data[:,6]

array([ 2.,  3.,  3.,  2.,  4.,  4.,  2.,  4.,  2.,  2.,  2.,  4.,  2.,
        4.,  2.,  2.,  3.,  3.,  3.,  3.,  2.,  2.,  2.,  4.,  2.,  4.,
        2.,  2.,  2.,  2.,  2.,  4.,  2.,  4.,  2.,  3.,  3.,  3.,  3.,
        3.,  3.,  2.,  2.,  2.,  4.,  2.,  2.,  4.,  3.,  2.,  2.,  2.,
        4.,  2.,  4.,  2.,  4.,  2.,  2.,  4.,  2.,  2.,  2.,  4.,  3.,
        3.,  2.,  1.,  2.,  2.,  2.,  3.,  2.,  2.,  2.,  2.,  2.,  4.,
        2.,  2.,  2.,  2.,  1.,  2.,  2.,  4.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  4.,  2.,  2.,  2.,  2.,  2.,  2.,  3.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  1.,  3.,  2.,  3.,  2.,  4.,
        2.,  2.,  4.,  2.,  2.,  2.,  2.,  2.,  6.,  2.,  2.,  2.,  3.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  6.,  5.,
        6.,  2.,  2.,  3.,  2.,  2.,  2.,  2.,  2.,  3.,  4.,  4.,  5.,
        6.,  4.,  2.,  4.,  4.,  2.,  3.,  2.,  2.,  3.,  2.,  4.,  2.,
        2.,  3.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  4

In [13]:
sum(np.isnan(data).flatten())

4

In [14]:
# 결측치에 중위값을 입력하도록 설정
imputer = SimpleImputer(strategy='median')

In [15]:
imputer.fit(data)
data_trans = imputer.transform(data)

In [16]:
data_trans

array([[16.99,  1.01,  0.  , ...,  3.  ,  1.  ,  2.  ],
       [10.34,  1.66,  1.  , ...,  3.  ,  1.  ,  3.  ],
       [21.01,  3.5 ,  1.  , ...,  3.  ,  1.  ,  3.  ],
       ...,
       [20.34,  1.66,  1.  , ...,  3.  ,  1.  ,  3.  ],
       [13.23,  2.66,  1.  , ...,  2.  ,  1.  ,  2.  ],
       [26.34,  2.2 ,  0.  , ...,  1.  ,  0.  ,  4.  ]])

In [17]:
# array를 다시 DataFrame 형식으로 변경
tips_trans = pd.DataFrame(data_trans, columns=['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'])

In [18]:
# 결측치 없어졌는지 확인
tips_trans.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [19]:
tips_trans

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,0.0,0.0,3.0,1.0,2.0
1,10.34,1.66,1.0,0.0,3.0,1.0,3.0
2,21.01,3.50,1.0,0.0,3.0,1.0,3.0
3,23.68,3.31,1.0,0.0,3.0,1.0,2.0
4,24.59,3.61,0.0,0.0,3.0,1.0,4.0
...,...,...,...,...,...,...,...
243,18.78,3.00,0.0,0.0,0.0,1.0,2.0
244,16.99,2.83,0.0,0.0,3.0,1.0,2.0
245,20.34,1.66,1.0,0.0,3.0,1.0,3.0
246,13.23,2.66,1.0,1.0,2.0,1.0,2.0


## 특징 추출 - RFE 차원 축소

In [20]:
# 목표변수 설정: total_bill(매출)
y = tips_trans['total_bill'] # 목표변수를 y에 지정하고
X = tips_trans.drop('total_bill', axis=1) # X(독립변수)에서는 제거

In [21]:
# 확인
X.describe()

,tip,sex,smoker,day,time,size
count,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000
mean,2.987621,0.641129,0.379032,1.733871,0.721774,2.572581
std,1.376134,0.480639,0.486127,1.153125,0.449031,0.949523
min,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2.000000,0.000000,0.000000,0.750000,0.000000,2.000000
50%,2.830000,1.000000,0.000000,2.000000,1.000000,2.000000
75%,3.520000,1.000000,1.000000,3.000000,1.000000,3.000000
max,10.000000,1.000000,1.000000,3.000000,1.000000,6.000000


In [22]:
y.describe()

count    248.000000
mean      19.776895
std        8.851566
min        3.070000
25%       13.347500
50%       17.795000
75%       24.127500
max       50.810000
Name: total_bill, dtype: float64

In [23]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVR

In [24]:
estimator = SVR(kernel="linear")
rfe = RFE(estimator, n_features_to_select=4)

In [25]:
rfe.fit(X, y)

RFE(estimator=SVR(kernel='linear'), n_features_to_select=4)

In [26]:
# summarize all features
for i in range(X.shape[1]):
  print('Column: %d, Selected=%s, Rank: %d' % (i, rfe.support_[i], rfe.ranking_[i]))

Column: 0, Selected=True, Rank: 1
Column: 1, Selected=False, Rank: 3
Column: 2, Selected=True, Rank: 1
Column: 3, Selected=False, Rank: 2
Column: 4, Selected=True, Rank: 1
Column: 5, Selected=True, Rank: 1


-> 매출이 목표변수일 때 
'tip', 'smoker', 'time', 'size'의 네 변수를 추출.

## 데이터 정규화 및 표준화
### MinMaxScaler

In [27]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [28]:
# 정규화
trans = MinMaxScaler()
tips_N = trans.fit_transform(X)
tips_norm = pd.DataFrame(tips_N, columns=['tip', 'sex', 'smoker', 'day', 'time', 'size'])
tips_norm.describe()

,tip,sex,smoker,day,time,size
count,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000
mean,0.220847,0.641129,0.379032,0.577957,0.721774,0.314516
std,0.152904,0.480639,0.486127,0.384375,0.449031,0.189905
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.111111,0.000000,0.000000,0.250000,0.000000,0.200000
50%,0.203333,1.000000,0.000000,0.666667,1.000000,0.200000
75%,0.280000,1.000000,1.000000,1.000000,1.000000,0.400000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [32]:
# 표준화
sc = StandardScaler()
tips_S = sc.fit_transform(X)
tips_stan = pd.DataFrame(tips_S, columns=['tip', 'sex', 'smoker', 'day', 'time', 'size'])
tips_stan.describe().round()

,tip,sex,smoker,day,time,size
count,248.0,248.0,248.0,248.0,248.0,248.0
mean,0.0,0.0,0.0,-0.0,-0.0,-0.0
std,1.0,1.0,1.0,1.0,1.0,1.0
min,-1.0,-1.0,-1.0,-2.0,-2.0,-2.0
25%,-1.0,-1.0,-1.0,-1.0,-2.0,-1.0
50%,-0.0,1.0,-1.0,0.0,1.0,-1.0
75%,0.0,1.0,1.0,1.0,1.0,0.0
max,5.0,1.0,1.0,1.0,1.0,4.0


## 특징 추출 - PCA 차원 축소

In [35]:
from sklearn.decomposition import PCA

In [38]:
trans = PCA(n_components=4)
X_dim = trans.fit_transform(tips_trans)
X_dim[:3,:]

array([[-2.98500905, -1.14716027,  1.78842197,  0.01180337],
       [-9.44192098, -1.5997611 ,  0.23424027,  1.04198106],
       [ 1.33645028, -1.33819469, -0.34697112,  0.2621155 ]])

## 특징 추출 - Regression Feature Selection
(Numerical Input, Numerical Output)

In [39]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

In [40]:
y = tips_trans['total_bill'] # 목표변수 설정
X = tips_trans.drop('total_bill', axis=1)

In [44]:
fs = SelectKBest(score_func=f_regression, k=4)
tips_RFS = fs.fit_transform(X, y)
tips_RFS[:3,:]

array([[1.01, 3.  , 1.  , 2.  ],
       [1.66, 3.  , 1.  , 3.  ],
       [3.5 , 3.  , 1.  , 3.  ]])

In [45]:
tips_trans

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,0.0,0.0,3.0,1.0,2.0
1,10.34,1.66,1.0,0.0,3.0,1.0,3.0
2,21.01,3.50,1.0,0.0,3.0,1.0,3.0
3,23.68,3.31,1.0,0.0,3.0,1.0,2.0
4,24.59,3.61,0.0,0.0,3.0,1.0,4.0
...,...,...,...,...,...,...,...
243,18.78,3.00,0.0,0.0,0.0,1.0,2.0
244,16.99,2.83,0.0,0.0,3.0,1.0,2.0
245,20.34,1.66,1.0,0.0,3.0,1.0,3.0
246,13.23,2.66,1.0,1.0,2.0,1.0,2.0
